<a href="https://colab.research.google.com/github/Joracosu/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG1/Jos%C3%A9_Ram%C3%B3n_Conde_AG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Actividad Guiada 1 de Algoritmos de Optimización

José Ramón Conde

https://colab.research.google.com/drive/1b0od-aP07ucKhmEqv2oQXfaofBG7L6km

In [4]:
def torres (N, desde, hasta):
  if N==1:
    print('Lleva la ficha desde', str(desde),'hasta',str(hasta))
  else:
    torres(N-1, desde, 6-desde-hasta)
    print('Lleva la ficha desde', str(desde), 'hasta', str(hasta))
    torres(N-1, 6-desde-hasta, hasta)

torres(4, 1, 3)

Lleva la ficha desde 1 hasta 2
Lleva la ficha desde 1 hasta 3
Lleva la ficha desde 2 hasta 3
Lleva la ficha desde 1 hasta 2
Lleva la ficha desde 3 hasta 1
Lleva la ficha desde 3 hasta 2
Lleva la ficha desde 1 hasta 2
Lleva la ficha desde 1 hasta 3
Lleva la ficha desde 2 hasta 3
Lleva la ficha desde 2 hasta 1
Lleva la ficha desde 3 hasta 1
Lleva la ficha desde 2 hasta 3
Lleva la ficha desde 1 hasta 2
Lleva la ficha desde 1 hasta 3
Lleva la ficha desde 2 hasta 3
